# 3.3.1. Generating the Dataset

In [2]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

/Users/vaibhavsethia07/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
class SyntheticRegressionData(d2l.DataModule):  #@save
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=1000, batch_size=32):
        super().__init__()
        self.save_hyperparameters()

        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1)*noise
        self.y = self.X@w.reshape((-1, 1)) + b + noise

data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)

In [4]:
print('features:', data.X[0], '\nlabel:', data.y[0])

features: tensor([-2.7652,  1.4419]) 
label: tensor([-6.2379])


# 3.3.2. Reading the Dataset

In [9]:
@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train=True):
    if train:
        indices = list[int](range(0, self.num_train))
        random.shuffle(indices)
    else:
        indices = list[int](range(self.num_train, self.num_train+self.num_val))
    
    for i in range(0, len(indices), self.batch_size):
        batch_indices = torch.tensor(indices[i:i+self.batch_size])
        yield self.X[batch_indices], self.y[batch_indices]


In [10]:
X, y = next(iter(data.get_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


# 3.3.3. Concise Implementation of the Data Loader

In [13]:
@d2l.add_to_class(SyntheticRegressionData)
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    dataset = torch.utils.data.TensorDataset(*tensors)
    return torch.utils.data.DataLoader(dataset, self.batch_size, shuffle=train)

@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train=True):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)

In [14]:
X, y = next(iter(data.get_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


In [15]:
len(data.get_dataloader())

32